In [1]:
import requests
import sys
import time
import numpy as np
from fastai.vision import parallel

Big Buck Bunny: 12 min 1080p 30fps.

In [2]:
test_url = "http://distribution.bbb3d.renderfarming.net/video/mp4/bbb_sunflower_1080p_30fps_normal.mp4"

A smaller video I found for testing.

In [3]:
test_url_2 = "https://cdn.videvo.net/videvo_files/video/premium/video0036/small_watermarked/cog_clock17_preview.mp4"

In [4]:
time_1 = time.time()

In [5]:
time.time() - time_1

0.011686563491821289

It's in seconds--Linux epoch divided by 1000.

In [6]:
# !wget 'http://distribution.bbb3d.renderfarming.net/video/mp4/bbb_sunflower_1080p_30fps_normal.mp4' -O bbb.mp4

Nice one-liner from https://stackoverflow.com/questions/2017843/fetch-frame-count-with-ffmpeg.

In [7]:
# !ffprobe -v error -select_streams v:0 -show_entries stream=nb_frames -of default=nokey=1:noprint_wrappers=1 -ignore_editlist 1 bbb.mp4

In [8]:
frame_count = 19036

In [9]:
fps = 30

In [10]:
minutes = frame_count / fps / 60
minutes

10.575555555555555

In [11]:
mb_count = 276134947 / (1e+6)

In [12]:
frame_size = mb_count / frame_count
frame_size

0.01450593333683547

`frame_size` represents the average MB size of frame.

In [13]:
second_size = frame_size * fps
second_size

0.4351780001050641

`second_size` represents the average MB size of a second at 30 FPS.

In [14]:
(round(0.2555 * 10000)/100)

25.55

In [15]:
using_url = test_url

In [16]:
process_count = 4

In [17]:
from multiprocessing import Process, Queue

mbps_queue = Queue()
mbps_percent_queue = Queue()

def download_measure(i):
    global mbps_list
    response = requests.get(using_url, stream=True)
    total_length = response.headers.get('content-length')

    start = time.time()
    dl = 0
    total_length = int(total_length)
    last_print_time = 0
    for data in response.iter_content(chunk_size=1024):
        dl += len(data)
        done = int(50 * dl / total_length)
        dl_mb = dl/(1e+6)
        if time.time() - last_print_time > 0.2:
            last_print_time = time.time()
            mbps_percent_queue.put((i, f"{i}: {round((dl/total_length) * 10000)/100}%"))
        
    elapsed = time.time() - start
    mbps_queue.put((total_length/1e+6)/elapsed)

In [18]:
processes = [Process(target=download_measure, args=(i,)) for i in range(process_count)]

In [19]:
from queue import Empty

def print_progress(): 
    try:
        progress_outputs = sorted([mbps_percent_queue.get(timeout=.2) for _ in processes], key=lambda a: a[0] if a else 0)
        out_str = ', '.join([p[1] for p in filter(lambda p: p, progress_outputs)])
        sys.stdout.write(f"\r{' ' * len(out_str)}")
        sys.stdout.write(f"\r{out_str}")
        sys.stdout.flush()  
    except Empty:
        pass

In [20]:
# Run processes
for p in processes:
    p.start()

#Exit the completed processes
while True in [p.is_alive() for p in processes]:
    print_progress()
    
# for p in processes:
#     p.join()

1: 99.98%, 2: 98.87%, 2: 98.89%, 2: 98.9%%

In [21]:
mbps_averages = [mbps_queue.get() for p in processes]
mbps_averages

[0.14129937461643993,
 0.13485796750720644,
 0.12793888819084998,
 0.1271423812682435]

In [22]:
mbps_average = np.average(mbps_averages)
mbps_average

0.13280965289568497

In [23]:
relative_length_ratio = second_size / mbps_average
relative_length_ratio

3.276704596516592

In [24]:
extra_length_proportion = relative_length_ratio - 1
extra_length_proportion

2.276704596516592

In [25]:
minutes_spent_buffering = extra_length_proportion * minutes
minutes_spent_buffering

24.077415944049914

That's an absurdly high number.